In [1]:
library(dplyr)
library(sf)
library(ggplot2)
library(units)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Linking to GEOS 3.9.1, GDAL 3.2.2, PROJ 8.0.0

udunits database from /usr/share/udunits/udunits2.xml



# genetic diversity of each marine species

In [6]:
master_matrices <- read.csv("master_matrices.csv",header=T) # πvalue between each two sequence
gendivers <- subset(master_matrices,seq1 > 0 & num_per_bp < 0.1 & overlap >= 0.6) %>% group_by(species) %>% 
  summarise(genetic_diversity=mean(num_per_bp),sd=sd(num_per_bp),seq_count=max(seq2),variation=sd/genetic_diversity) %>% 
    subset(seq_count > 3) # genetic diversity of each species (select 0 <πvalue < 0.1,and overlap between two sequence should >= 60%)
gendivers$species <- gendivers$species %>% as.character() %>% strsplit(split = "_") %>%
  lapply(function(x){paste(x[1],x[2],sep=" ")}) %>% unlist()   # species name list of genetic diversity
length(unique(gendivers$species))
#data filter
gendivers_corrected <- subset(gendivers,variation < 3) # delete abnormal genetic diversity value>outlier: genetic_diversity < outlier & 
write.table(gendivers_corrected$species,"genetic_data_species_list.txt",sep="\n",col.names=F,row.names=F,quote=F)
write.table(gendivers_corrected,"gendivers_corrected.csv",sep = ',',row.names=F)

[1] 4417

In [8]:
gendivers_corrected <- read.csv("gendivers_corrected.csv") # genetic diversity for each species
gendivers_corrected$species <- gsub('_',' ',gendivers_corrected$species)
species_grid_filtered <- read.csv("species_distribution_grid_filtered.csv",col.names=c('species','grid_id','long','lat','area')) #species distribution
ocean_grid_id <- read.csv("ocean_grid_id.csv") %>% pull(grid_id)

equal_area_grid <- sf::st_read(dsn = "SHP//Export_Output_5.shp")
genetic_diversity_pattern <- filter(species_grid_filtered, grid_id %in% ocean_grid_id) %>% #  & area > 385900*385900*0.5
    right_join(gendivers_corrected,by ='species') # merge species genetic diversity and distribution
summary(genetic_diversity_pattern)
length(unique(genetic_diversity_pattern$species))
filter(genetic_diversity_pattern,!is.na(grid_id)) %>% pull(species) %>% unique() %>% length()

Reading layer `Export_Output_5' from data source `/home/fhz/users/HMP/conservation_marine_animals/GD/git_hub/03.map/SHP/Export_Output_5.shp' using driver `ESRI Shapefile'
Simple feature collection with 3458 features and 10 fields
geometry type:  MULTIPOLYGON
dimension:      XY
bbox:           xmin: -180 ymin: -85.22194 xmax: 180 ymax: 89.97895
geographic CRS: StandardWorldMapCoordination


   species             grid_id          long                lat          
 Length:793918      Min.   :   1   Min.   :-17174579   Min.   :-7123536  
 Class :character   1st Qu.:1224   1st Qu.: -6369379   1st Qu.:-2106836  
 Mode  :character   Median :1694   Median :  5593521   Median : -177336  
                    Mean   :1695   Mean   :  3113655   Mean   : -162956  
                    3rd Qu.:2166   3rd Qu.: 12539721   3rd Qu.: 1752164  
                    Max.   :3457   Max.   : 17170521   Max.   : 7154764  
                    NA's   :230    NA's   :230         NA's   :230       
      area           genetic_diversity         sd             seq_count     
 Min.   :0.000e+00   Min.   :0.0002045   Min.   :0.000000   Min.   :  4.00  
 1st Qu.:3.110e+10   1st Qu.:0.0039543   1st Qu.:0.002455   1st Qu.:  7.00  
 Median :8.340e+10   Median :0.0069744   Median :0.005426   Median : 12.00  
 Mean   :8.431e+10   Mean   :0.0124713   Mean   :0.011000   Mean   : 23.16  
 3rd Qu.:1.489e+11   3r

[1] 4316

[1] 4086

# genetic diversity of each cell

In [10]:
data <- genetic_diversity_pattern %>% group_by(grid_id) %>%
    summarise(co1_genetic_diversity_mean=mean(genetic_diversity),co1_sampled_species_richness=length(unique(species))) # average species genetic diversity in each grid
map <- left_join(data,equal_area_grid,by = 'grid_id')
data_table <- dplyr::select(map,c(grid_id,long,lat,co1_genetic_diversity_mean,co1_sampled_species_richness))
write.csv(data_table,"data_table.csv",row.names=F)